In [1]:
import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [2]:
test_data = pd.read_csv('../DATA/test_normalized_4labels.csv', index_col=0)
training_data = pd.read_csv(
    '../DATA/train_normalized_4labels.csv', index_col=0)


/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
labels = {'CERTIFIED': [1, 0, 0, 0], 'CERTIFIED-WITHDRAWN': [0, 1, 0, 0],
          'DENIED': [0, 0, 1, 0], 'WITHDRAWN': [0, 0, 0, 1]}
test_data['CASE_STATUS'] = test_data['CASE_STATUS'].apply(lambda x: labels[x])
training_data['CASE_STATUS'] = training_data['CASE_STATUS'].apply(
    lambda x: labels[x])


In [4]:
train_data, val_data = train_test_split(training_data, test_size=0.2)


In [5]:
train_labels = np.array(train_data.pop('CASE_STATUS'))
val_labels = np.array(val_data.pop('CASE_STATUS'))
test_labels = np.array(test_data.pop('CASE_STATUS'))

train_data.pop('EMPLOYER_NAME')
test_data.pop('EMPLOYER_NAME')
val_data.pop('EMPLOYER_NAME')

train_features = np.array(train_data)
val_features = np.array(val_data)
test_features = np.array(test_data)


In [ ]:
print(train_features.shape)
print(test_features.shape)
print(val_features.shape)
print(train_labels.shape)
print(test_labels.shape)
print(val_labels.shape)

(1363656, 420)
(730531, 420)
(340915, 420)
(1363656,)
(730531,)
(340915,)


In [6]:
METRICS = [
    keras.metrics.CategoricalAccuracy(name="categorical_accuracy")
]

def make_model(metrics=METRICS, output_bias=None):
  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)
  model = keras.Sequential([
      keras.layers.Dense(
          256, activation='relu',
          input_shape=(train_features.shape[-1],)),
      keras.layers.Dense(
          64, activation='relu'),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(4, activation='sigmoid',
                         bias_initializer=output_bias),
  ])

  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=1e-3),
      loss=keras.losses.CategoricalCrossentropy(),
      metrics=metrics)

  return model


2021-12-05 07:07:27.731883: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
EPOCHS = 100
BATCH_SIZE = 2048
filename = '../saved_models/nn_2labels_model.sav'
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=filename,
                                                 save_weights_only=True,
                                                 verbose=1)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_categorical_accuracy',
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)
model = make_model()
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               107776    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                16448     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 260       
Total params: 124,484
Trainable params: 124,484
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.fit(
    train_features,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(val_features, val_labels),
    callbacks=[early_stopping, cp_callback],
    verbose=1)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

In [ ]:
model.evaluate(test_features, test_labels)

22830/22830 [==============================] - 34s 1ms/step - loss: 0.0666 - tp: 74403.0000 - fp: 1199.0000 - tn: 642816.0000 - fn: 12113.0000 - accuracy: 0.9818 - precision: 0.9841 - recall: 0.8600 - auc: 0.9812 - prc: 0.9513


[0.06662300229072571,
 74403.0,
 1199.0,
 642816.0,
 12113.0,
 0.9817776679992676,
 0.9841406345367432,
 0.859991192817688,
 0.9812482595443726,
 0.9512678384780884]

In [ ]:
model.evaluate(train_features, train_labels)


42615/42615 [==============================] - 67s 2ms/step - loss: -850.4210 - tp: 95380.0000 - fp: 50571.0000 - tn: 1151701.0000 - fn: 66004.0000 - accuracy: 0.8933 - precision: 0.6535 - recall: 0.5910 - auc: 0.9017 - prc: 0.5666


[-850.4209594726562,
 95380.0,
 50571.0,
 1151701.0,
 66004.0,
 0.8932589888572693,
 0.6535069942474365,
 0.591012716293335,
 0.9016558527946472,
 0.5666338801383972]